## DX799S O1 Data Science Capstone (Summer 1 2025): ACTIVITY 1.2 ##

Each week, you will apply the concepts of that week to your Integrated Capstone Project’s dataset. In preparation for Milestone One, create a Jupyter Notebook (similar to in Module B, semester two) that illustrates these lessons. There are no specific questions to answer in your Jupyter Notebook files in this course; your general goal is to analyze your data, using the methods you have learned about in this course and in this program, and draw interesting conclusions. 

For Week 1, include concepts such as linear regression with polynomial terms, interaction terms, multicollinearity, variance inflation factor and regression, and categorical and continuous features. Complete your Jupyter Notebook homework by 11:59 pm ET on Sunday. 

In Week 7, you will compile your findings from your Jupyter Notebook homework into your Milestone One assignment for grading. For full instructions and the rubric for Milestone One, refer to the following link. 

In [43]:
# ===================================
# Useful Imports: Add more as needed
# ===================================

# Standard Libraries
import os
import time
import math
import io
import zipfile
import requests
from urllib.parse import urlparse
from itertools import chain, combinations

# Data Science Libraries
import numpy as np
import pandas as pd
import seaborn as sns

# Visualization
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.ticker as mticker  # Optional: Format y-axis labels as dollars
import seaborn as sns

# Scikit-learn (Machine Learning)
from sklearn.model_selection import (
    train_test_split, 
    cross_val_score, 
    GridSearchCV, 
    RandomizedSearchCV, 
    RepeatedKFold
)
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.feature_selection import SequentialFeatureSelector, f_regression, SelectKBest
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score



The following dataset, "Video Review", is a collection of information that was created based on reviewable video evidence that outlines the events that resulted in a concussion during punt players in the NFL 2016-2017 season. The target, Primary_Impact_Type, outlines if the concussion occurred from the impact of Helmet-to-Helmet, Helmet-to-Body, or Helmet-to-Ground.

In [44]:
#Video Review Dataset 

df_videoreview = pd.read_csv("video_review.csv")

label_encoder = LabelEncoder()

print("Object columns before encoding:")
print(df_videoreview.select_dtypes(include=['object']).columns)

for col in df_videoreview.select_dtypes(include=['object']).columns:
    df_videoreview[col] = label_encoder.fit_transform(df_videoreview[col].astype(str))


target_column = 'Primary_Impact_Type'  
X = df_videoreview.drop(columns=[target_column])
y = df_videoreview[target_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



Object columns before encoding:
Index(['Player_Activity_Derived', 'Turnover_Related', 'Primary_Impact_Type',
       'Primary_Partner_GSISID', 'Primary_Partner_Activity_Derived',
       'Friendly_Fire'],
      dtype='object')


In [45]:
#Video Review linear regression with categorical and continuous features

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)

from sklearn.metrics import mean_squared_error, r2_score
print("MSE Video Review:", mean_squared_error(y_train, y_pred_train))
print("R² Video Review:", r2_score(y_train, y_pred_train))


MSE Video Review: 0.7820389432809272
R² Video Review: 0.2676004996667486


In [46]:
#Video Review linear regression with polynomial terms

poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)

model = LinearRegression()
model.fit(X_train_poly, y_train)

mse_scores = cross_val_score(model, X_train_poly, y_train, 
                             scoring='neg_mean_squared_error', cv=5)

mse_scores = -mse_scores  
avg_mse = np.mean(mse_scores)

print("Cross-Validated MSE Scores of Video Review with Polynomial Terms:", mse_scores)
print("Average Cross-Validated MSE:", avg_mse)


Cross-Validated MSE Scores of Video Review with Polynomial Terms: [14.24508986  6.8337424   2.64275896  6.42421274  5.88742934]
Average Cross-Validated MSE: 7.206646660414441


In [47]:
#Video Review linear regression with interaction terms

interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_train_interact = interaction.fit_transform(X_train_scaled)

model = LinearRegression()
model.fit(X_train_interact, y_train)

y_train_pred = model.predict(X_train_interact)
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)


cv_mse_scores = -cross_val_score(model, X_train_interact, y_train,
                                  scoring='neg_mean_squared_error', cv=5)

print("Cross-Validated MSE Scores (Interaction Terms) Video Revoew:", cv_mse_scores)
print("Average Cross-Validated MSE (Interaction Terms) Video Review:", np.mean(cv_mse_scores))


Cross-Validated MSE Scores (Interaction Terms) Video Revoew: [6.09283055 8.01985695 5.65582245 7.49766965 7.63417845]
Average Cross-Validated MSE (Interaction Terms) Video Review: 6.980071608978699


In [48]:
#Video Review linear regression with multicollinearity

model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_pred_train = model.predict(X_train_scaled)

mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("MSE Video Review with multicollinearity:", mse_train)
print("R² Video Review with multicollinearity:", r2_train)

coef_df = pd.DataFrame({'Feature': X.columns, 'Coefficient': model.coef_})
print("Coefficients:")
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))


MSE Video Review with multicollinearity: 0.7820389432809272
R² Video Review with multicollinearity: 0.2676004996667486
Coefficients:
                            Feature   Coefficient
1                           GameKey  5.017196e-01
0                       Season_Year -4.723777e-01
4           Player_Activity_Derived -4.587435e-01
8                     Friendly_Fire -4.444616e-01
7  Primary_Partner_Activity_Derived  3.349750e-01
3                            GSISID  2.988874e-01
6            Primary_Partner_GSISID  1.899683e-01
2                            PlayID -7.620664e-02
5                  Turnover_Related  2.428613e-17


In [49]:
!pip install statsmodels

In [50]:
#Video Review linear regression with variance inflation factor and regression
from statsmodels.stats.outliers_influence import variance_inflation_factor

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

def calculate_vif(X_df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_df.columns
    vif_data["VIF"] = [variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])]
    return vif_data

vif_df = calculate_vif(X_train_scaled_df)
print("Initial VIF values:")
print(vif_df.sort_values(by="VIF", ascending=False))

high_vif_features = vif_df[vif_df["VIF"] > 10]["feature"].tolist()
X_filtered = X_train_scaled_df.drop(columns=high_vif_features)


model = LinearRegression()
model.fit(X_filtered, y_train)
y_pred_train = model.predict(X_filtered)

mse = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

print("MSE of Video Review with VIF:", mse)
print("R² Video Review with VIF:", r2)

coef_df = pd.DataFrame({'Feature': X_filtered.columns, 'Coefficient': model.coef_})
print("\nCoefficients (Reduced Model):")
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))


Initial VIF values:
                            feature       VIF
0                       Season_Year  4.689585
1                           GameKey  3.588601
8                     Friendly_Fire  2.387368
3                            GSISID  2.289465
7  Primary_Partner_Activity_Derived  2.199185
2                            PlayID  1.422762
6            Primary_Partner_GSISID  1.415988
4           Player_Activity_Derived  1.304255
5                  Turnover_Related       NaN
MSE of Video Review with VIF: 0.7820389432809272
R² Video Review with VIF: 0.2676004996667486

Coefficients (Reduced Model):
                            Feature   Coefficient
1                           GameKey  5.017196e-01
0                       Season_Year -4.723777e-01
4           Player_Activity_Derived -4.587435e-01
8                     Friendly_Fire -4.444616e-01
7  Primary_Partner_Activity_Derived  3.349750e-01
3                            GSISID  2.988874e-01
6            Primary_Partner_GSISID  1.899683

/home/codespace/.python/current/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


The next dataset, Injury Record, looks to determine the relationship between the playing surface and the injury and performance of NFL athletes. The Injury Record dataset accounts for 105 lower-limbs injuries that occurred over two seasons during the regular NFL season and provides information on the surface the game occurred on and the number of days the player missed due to injury (or how severe it was). The target in this case is surface which lists the type of surface (synethic or natural) the field was when the injury occurred.

In [51]:
df_injuryrecord = pd.read_csv("InjuryRecord.csv")

In [52]:
label_encoder = LabelEncoder()

print("Object columns before encoding:")
print(df_injuryrecord.select_dtypes(include=['object']).columns)

for col in df_injuryrecord.select_dtypes(include=['object']).columns:
    df_injuryrecord[col] = label_encoder.fit_transform(df_injuryrecord[col].astype(str))


target_column = 'Surface'  
X_injury = df_injuryrecord.drop(columns=[target_column])
y_injury = df_injuryrecord[target_column]

X_train, X_test, y_train, y_test = train_test_split(X_injury, y_injury, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Object columns before encoding:
Index(['GameID', 'PlayKey', 'BodyPart', 'Surface'], dtype='object')


In [53]:
#Injury Record linear regression with categorical and continuous features

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)

from sklearn.metrics import mean_squared_error, r2_score
print("MSE Injury Record:", mean_squared_error(y_train, y_pred_train))
print("R² Injury Record:", r2_score(y_train, y_pred_train))


MSE Injury Record: 0.23365898721080608
R² Injury Record: 0.06057104629091281


In [54]:
#Injury Record linear regression with polynomial terms
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)

model = LinearRegression()
model.fit(X_train_poly, y_train)

mse_scores_injury= cross_val_score(model, X_train_poly, y_train, 
                             scoring='neg_mean_squared_error', cv=5)

mse_scores_injury = -mse_scores_injury  
avg_mse_injury = np.mean(mse_scores_injury)

print("Cross-Validated MSE Scores of Video Review with Polynomial Terms:", mse_scores_injury)
print("Average Cross-Validated MSE:", avg_mse_injury)


Cross-Validated MSE Scores of Video Review with Polynomial Terms: [0.47917844 0.43635777 0.68074116 1.65513593 7.93238677]
Average Cross-Validated MSE: 2.2367600129354925


In [55]:
#Injury Record linear regression with interaction terms

interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_train_interact = interaction.fit_transform(X_train_scaled)

model = LinearRegression()
model.fit(X_train_interact, y_train)

y_train_pred = model.predict(X_train_interact)
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

cv_mse_scores_injury = -cross_val_score(model, X_train_interact, y_train,
                                  scoring='neg_mean_squared_error', cv=5)

print("Cross-Validated MSE Scores (Interaction Terms) Injury Record:", cv_mse_scores_injury)
print("Average Cross-Validated MSE (Interaction Terms) Injury Record:", np.mean(cv_mse_scores_injury))

Cross-Validated MSE Scores (Interaction Terms) Injury Record: [0.41871417 0.41943428 0.52558763 0.52013792 0.67010958]
Average Cross-Validated MSE (Interaction Terms) Injury Record: 0.5107967165732716


In [56]:
#Injury Record linear regression with multicollinearity

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)

mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("MSE Injury Record with multicollinearity:", mse_train)
print("R² Injury Record with multicollinearity:", r2_train)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

coef_df = pd.DataFrame({
    'Feature': X_train_scaled_df.columns,
    'Coefficient': model.coef_
})

print("Coefficients:")
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))


MSE Injury Record with multicollinearity: 0.23365898721080608
R² Injury Record with multicollinearity: 0.06057104629091281
Coefficients:
     Feature   Coefficient
0  PlayerKey  2.895148e-01
1     GameID -2.254470e-01
6     DM_M28  1.380293e-01
7     DM_M42 -9.425601e-02
2    PlayKey  5.508942e-02
3   BodyPart  2.208297e-03
5      DM_M7 -3.225481e-04
4      DM_M1  1.318390e-16


In [57]:
#Injury Record linear regression with variance inflation factor and regression
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

def calculate_vif(X_df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_df.columns
    vif_data["VIF"] = [variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])]
    return vif_data

vif_df = calculate_vif(X_train_scaled_df)
print("Initial VIF values:")
print(vif_df.sort_values(by="VIF", ascending=False))

high_vif_features = vif_df[vif_df["VIF"] > 10]["feature"].tolist()
X_filtered = X_train_scaled_df.drop(columns=high_vif_features)


model = LinearRegression()
model.fit(X_filtered, y_train)
y_pred_train = model.predict(X_filtered)

mse = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

print("MSE of Injury Record with VIF:", mse)
print("R² Injury Record with VIF:", r2)

coef_df = pd.DataFrame({'Feature': X_filtered.columns, 'Coefficient': model.coef_})
print("\nCoefficients (Reduced Model):")
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))

Initial VIF values:
     feature        VIF
1     GameID  19.718543
0  PlayerKey  17.941774
6     DM_M28   3.368880
7     DM_M42   3.080271
2    PlayKey   1.553218
5      DM_M7   1.431353
3   BodyPart   1.063383
4      DM_M1        NaN
MSE of Injury Record with VIF: 0.24156478051584446
R² Injury Record with VIF: 0.02878570295168159

Coefficients (Reduced Model):
    Feature   Coefficient
4    DM_M28  1.147376e-01
5    DM_M42 -7.608983e-02
0   PlayKey  6.504045e-02
1  BodyPart -2.066456e-03
3     DM_M7  1.101684e-03
2     DM_M1 -2.151057e-16


/home/codespace/.python/current/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss


The last dataset, Concussion, contains a list of concussion injuries that occurred in the National Football League from the year 2012 to 2014. The data includes features such as Position, Pre-Season Injury?, Week of Injury, Weeks Injured, Games Missed, Reported Injury Type, Average Playtime Before Injury, etc. The target in this case will be "Reported Injury Type" which will be limited to just concussions.

In [58]:
df_concussion = pd.read_csv("Concussion Injuries 2012-2014 (1).csv")


In [59]:
df_clean_concussion = df_concussion.drop(columns=['ID', 'Player', 'Game', 'Date', 'Winning Team?', 'Unknown Injury?'])
df_clean_concussion = df_clean_concussion.dropna()

df_clean_concussion.head()

,Team,Opposing Team,Position,Pre-Season Injury?,Week of Injury,Season,Weeks Injured,Games Missed,Reported Injury Type,Total Snaps,Play Time After Injury,Average Playtime Before Injury
0,Washington Redskins,Tampa Bay Buccaneers,Wide Receiver,No,4,2012/2013,1,1.0,Head,0,14 downs,37.00 downs
1,San Diego Chargers,Tennessee Titans,Offensive Tackle,No,3,2013/2014,1,1.0,Concussion,0,78 downs,73.50 downs
2,Buffalo Bills,Houston Texans,Wide Receiver,No,4,2014/2015,1,1.0,Concussion,0,25 downs,17.50 downs
3,New England Patriots,Buffalo Bills,Center,No,6,2014/2015,1,1.0,Head,0,82 downs,41.50 downs
16,San Francisco 49ers,Arizona Cardinals,Guard,No,8,2012/2013,1,0.0,Concussion,1,8 downs,14.43 downs


In [60]:
label_encoder = LabelEncoder()


for col in df_clean_concussion.select_dtypes(include=['object']).columns:
    df_clean_concussion[col] = label_encoder.fit_transform(df_clean_concussion[col].astype(str))

target_column = 'Reported Injury Type'  
X_concussion = df_clean_concussion.drop(columns=[target_column])
y_concussion = df_clean_concussion[target_column]

X_train, X_test, y_train, y_test = train_test_split(X_concussion, y_concussion, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [61]:
#Concussion linear regression with categorical and continuous features
model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)

from sklearn.metrics import mean_squared_error, r2_score
print("MSE Concussion:", mean_squared_error(y_train, y_pred_train))
print("R² Concussion:", r2_score(y_train, y_pred_train))

MSE Concussion: 0.12325581378002946
R² Concussion: 0.21830443648278552


In [62]:
#Concussion linear regression with polynomial terms
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

model = LinearRegression()
model.fit(X_train_poly, y_train)

mse_scores_concussion= cross_val_score(model, X_train_poly, y_train, 
                             scoring='neg_mean_squared_error', cv=5)

mse_scores_concussion = -mse_scores_concussion  
avg_mse_concussion = np.mean(mse_scores_concussion)

print("Cross-Validated MSE Scores of Video Review with Polynomial Terms:", mse_scores_concussion)
print("Average Cross-Validated MSE:", avg_mse_concussion)

Cross-Validated MSE Scores of Video Review with Polynomial Terms: [0.14010582 0.11294197 0.14327489 0.16049187 0.18380589]
Average Cross-Validated MSE: 0.14812408681002284


In [63]:
#Concussion linear regression with interaction terms

interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_train_interact = interaction.fit_transform(X_train_scaled)

model = LinearRegression()
model.fit(X_train_interact, y_train)

y_train_pred = model.predict(X_train_interact)
mse_train = mean_squared_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

cv_mse_scores_concussion = -cross_val_score(model, X_train_interact, y_train,
                                  scoring='neg_mean_squared_error', cv=5)

print("Cross-Validated MSE Scores (Interaction Terms) Concussion:", cv_mse_scores_concussion)
print("Average Cross-Validated MSE (Interaction Terms) Concussion:", np.mean(cv_mse_scores_concussion))

Cross-Validated MSE Scores (Interaction Terms) Concussion: [0.16483957 0.11429592 0.16082085 0.17825109 0.20125835]
Average Cross-Validated MSE (Interaction Terms) Concussion: 0.16389315859340267


In [64]:
#Concussion linear regression with multicollinearity

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred_train = model.predict(X_train_scaled)

mse_train = mean_squared_error(y_train, y_pred_train)
r2_train = r2_score(y_train, y_pred_train)

print("MSE Concussion with multicollinearity:", mse_train)
print("R² Concussion with multicollinearity:", r2_train)

X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

coef_df = pd.DataFrame({
    'Feature': X_train_scaled_df.columns,
    'Coefficient': model.coef_
})

print("Coefficients:")
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))


MSE Concussion with multicollinearity: 0.12325581378002946
R² Concussion with multicollinearity: 0.21830443648278552
Coefficients:
                           Feature   Coefficient
5                           Season -1.816350e-01
0                             Team -5.559658e-02
7                     Games Missed -4.219609e-02
4                   Week of Injury -3.436278e-02
9           Play Time After Injury -2.839721e-02
1                    Opposing Team  1.970363e-02
8                      Total Snaps  1.222037e-02
2                         Position -3.527860e-03
10  Average Playtime Before Injury  1.134226e-03
3               Pre-Season Injury?  1.040834e-17
6                    Weeks Injured -6.938894e-18


In [65]:
#Concussion linear regression with variance inflation factor and regression
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)

def calculate_vif(X_df):
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_df.columns
    vif_data["VIF"] = [variance_inflation_factor(X_df.values, i) for i in range(X_df.shape[1])]
    return vif_data

vif_df = calculate_vif(X_train_scaled_df)
print("Initial VIF values:")
print(vif_df.sort_values(by="VIF", ascending=False))

high_vif_features = vif_df[vif_df["VIF"] > 10]["feature"].tolist()
X_filtered = X_train_scaled_df.drop(columns=high_vif_features)


model = LinearRegression()
model.fit(X_filtered, y_train)
y_pred_train = model.predict(X_filtered)

mse = mean_squared_error(y_train, y_pred_train)
r2 = r2_score(y_train, y_pred_train)

print("MSE of Concussion with VIF:", mse)
print("R² Concussion with VIF:", r2)

coef_df = pd.DataFrame({'Feature': X_filtered.columns, 'Coefficient': model.coef_})
print("\nCoefficients (Reduced Model):")
print(coef_df.sort_values(by='Coefficient', key=abs, ascending=False))

Initial VIF values:
                           feature       VIF
10  Average Playtime Before Injury  1.686492
9           Play Time After Injury  1.502301
8                      Total Snaps  1.312588
7                     Games Missed  1.225132
4                   Week of Injury  1.195167
5                           Season  1.150839
2                         Position  1.093167
1                    Opposing Team  1.040517
0                             Team  1.035365
3               Pre-Season Injury?       NaN
6                    Weeks Injured       NaN
MSE of Concussion with VIF: 0.12325581378002946
R² Concussion with VIF: 0.21830443648278552

Coefficients (Reduced Model):
                           Feature   Coefficient
5                           Season -1.816350e-01
0                             Team -5.559658e-02
7                     Games Missed -4.219609e-02
4                   Week of Injury -3.436278e-02
9           Play Time After Injury -2.839721e-02
1                    Op

/home/codespace/.python/current/lib/python3.12/site-packages/statsmodels/regression/linear_model.py:1784: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.uncentered_tss
